# OpenAI 매개변수

# 개요
OpenAI 모델에 요청을 보낼 때 여러 매개변수를 사용하여 모델의 동작과 출력을 제어할 수 있습니다. \
이러한 매개변수를 이해하면 텍스트 생성, 질문 응답 또는 기타 사용 사례에 맞게 응답을 세부 조정할 수 있습니다.

더 자세한 예제는 공식 문서를 참조하세요: [Azure OpenAI Service](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference)


In [ ]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-12-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
SEED = 123

# 매개변수: max_tokens
**설명**: 생성할 응답의 최대 토큰 수를 설정합니다. \
**기본값**: 16 \
**예제**: max_tokens=50

프롬프트의 토큰 수와 max_tokens의 합은 모델의 컨텍스트 길이를 초과할 수 없습니다. \
gpt-4o-mini의 경우 16,384 tokens이며 모델에 따라 다릅니다. 

In [ ]:
def call_openai_with_max_tokens(max_tokens):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
                    {"role":"user","content": "최고의 반려동물은 "}],
                    max_tokens=max_tokens
    )
    return response.choices[0].message.content

tokens = [32, 64, 120, 200]
for token in tokens:
    print(f"Max Tokens: {token}\n")
    print(call_openai_with_max_tokens(token))
    print("\n" + "-"*80 + "\n")

# 매개변수: temperature

**설명**: 출력의 무작위성을 제어합니다. 낮은 값은 출력을 더 결정론적으로 만들고, 높은 값은 무작위성을 증가시킵니다. \
모델이 다음 토큰을 선택할 때, 여러 후보 중에서
- 높은 확률 토큰을 쓸지
- 조금 낮은 확률 토큰까지 선택할 여지를 줄지

**값 범위**: 0에서 2 \
**기본값**: 1 \
**예제**: temperature=0.7

높은 값은 모델이 더 창의적인 출력을 생성하도록 합니다. \
창의적인 응용 프로그램에는 0.9를, 명확한 답변이 필요한 경우에는 0(최대 확률 샘플링)을 시도하세요.

---
**참고**: 일반적으로 이 매개변수 또는 top_p를 조정하되 둘 다 동시에 조정하지 않는 것을 권장합니다.


In [ ]:
def call_openai(num_times, prompt, temperature=0.8, use_seed=False):
    for i in range(num_times):
        if use_seed:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    seed=SEED,
                    temperature = temperature
            )
        else:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages = [{"role":"system", "content":"You are a helpful assistant."},
                            {"role":"user","content": prompt}],
                    max_tokens=60,
                    temperature = temperature
            )
        print(response.choices[0].message.content)

In [ ]:
# Without seed and temperature, the response is different each time
call_openai(10, '최고의 반려동물은 ')

In [ ]:
# Now using a seed and 0 temperature, the response is the much more consisitent
call_openai(10, '최고의 반려동물은 ', temperature = 0, use_seed=True)

# 매개변수: n
**설명**: 각 프롬프트에 대해 생성할 응답의 수를 지정합니다. \
**기본값**: 1 \
**예제**: n = 3 

---
**참고**: 이 매개변수는 여러 응답을 생성하므로 토큰 할당량을 빠르게 소모할 수 있습니다. 신중하게 사용하고 max_tokens 및 stop 설정이 적절한지 확인하세요.

In [ ]:
response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages = [{"role":"system", "content":"You are a helpful assistant."},
                        {"role":"user","content": "최고의 반려동물은"}],
                max_tokens=60,
                n=2
        )

for index, c in enumerate(response.choices):
    print(index, c.message.content)
    print("\n" + "-"*80 + "\n")

# 매개변수: presence_penalty
**설명**: 텍스트에 이미 나타난 토큰을 기반으로 새 토큰에 페널티를 부여하여 모델이 새로운 토큰을 사용하도록 유도합니다. \
**값 범위**: -2.0에서 2.0 \
**기본값**: 0 \
**예제**: presence_penalty=0.5

**특징**  
- 단어가 *한 번이라도 등장*했으면 그 이후 사용 확률을 낮춤  
- "존재 여부" 기반이므로 반복 횟수와는 무관  
- 표현을 살짝 다양하게 하고 싶을 때 적합  
- 자연스러움을 크게 해치지 않으면서 중복을 줄이는 용도

**주의 사항**  
- 코드 생성·SQL·JSON·Tool Call처럼 **반복이 필수인 출력에서는 사용 금지**  
- 주제어가 반복될 수밖에 없는 설명형 프롬프트에서는 효과가 제한적  
- 값이 너무 높으면 문맥과 자연스러움이 손상될 수 있음

---

### presence_penalty vs frequency_penalty 비교

| 특징 | presence_penalty | frequency_penalty |
|------|------------------|-------------------|
| **기준** | 존재 여부 (0/1) | 반복 횟수 (누적) |
| **강도** | 약함 | 강함 |
| **효과** | 미묘한 변화 | 확실한 억제 |
| **용도** | 자연스러운 다양성 | 반복 패턴 제거 |
| **예시** | 일반 글쓰기 | 창의적 콘텐츠 |


In [ ]:
def call_openai_with_presence_penalty(presence_penalty):
    response = client.chat.completions.create(
          model=CHAT_COMPLETIONS_MODEL,
          messages = [{"role":"system", "content":"You are a helpful assistant."},
            {"role": "user", "content": "'고양이'에 대한 10문장을 작성해줘. 단, 가능한 자연스럽게 작성해줘."}
        ],
                    max_tokens=400,
                    presence_penalty=presence_penalty, 
                    
    )
    return response.choices[0].message.content

# Generate with different presence_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Presence Penalty: {penalty}\n")
    print(call_openai_with_presence_penalty(penalty))
    print("\n" + "-"*80 + "\n")

# 매개변수: frequency_penalty
**설명**: 텍스트에 이미 나타난 빈도를 기반으로 새 토큰에 페널티를 부여하여 동일한 줄을 반복할 가능성을 줄입니다. \
**값 범위**: -2.0에서 2.0 \
**기본값**: 0 \
**예제**: frequency_penalty=0.5

**특징**  
- 동일 단어가 반복될수록 페널티 강도가 증가  
- presence_penalty보다 훨씬 강한 억제 효과  
- 반복되는 문장·패턴을 확실하게 줄이고 싶을 때 적합  
- 창의적 글쓰기, 마케팅 문구, 스토리텔링에 유용

**주의 사항**  
- 변수명·키 이름·구문 반복이 중요한 **코드/SQL/JSON 생성에는 절대 사용하면 안 됨**  
- 높은 값일수록 다양성은 증가하지만 문맥 일관성은 낮아질 수 있음  
- 정보 전달이나 기술적 설명 같은 정확성이 필요한 응답에는 추천하지 않음

In [ ]:
def call_openai_with_frequency_penalty(frequency_penalty):
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "'고양이'에 대한 10문장을 작성해줘. 단, 가능한 자연스럽게 작성해줘."}
        ],
        max_tokens=400,
        frequency_penalty=frequency_penalty
    )
    return response.choices[0].message.content


# Generate with different frequency_penalty values
penalties = [0, 0.5, 1.0, 1.5, 2.0]
for penalty in penalties:
    print(f"Frequency Penalty: {penalty}\n")
    print(call_openai_with_frequency_penalty(penalty))
    print("\n" + "-"*80 + "\n")

### 탐색할 사용 사례
1. **응답 비교**  
   여러 응답을 생성하여 사용 사례에 가장 적합한 결과를 선택하세요.

2. **다양성 증가**  
   창의적 응용 프로그램에서 다양한 표현을 얻기 위해 여러 응답을 생성하세요.

3. **강건성 향상**  
   여러 응답을 생성하여 일관성과 정확성을 비교·검증하세요.

---

### 모범 사례
1. **프롬프트 길이 최적화**  
   프롬프트는 간결하지만 충분한 정보를 담도록 작성하세요.

2. **Temperature 및 Top_p 조정**  
   결정론적 vs. 창의적 응답의 균형을 위해 온도를 적절히 조정하세요.

3. **토큰 사용량 모니터링**  
   `max_tokens`를 적절히 설정하여 비용과 응답 길이를 관리하세요.

4. **중지 시퀀스 사용**  
   모델이 텍스트 생성을 중단해야 하는 지점을 정의해 출력을 안정적으로 제어하세요.

5. **여러 응답 생성**  
   `n` 매개변수를 사용하여 여러 응답을 생성하고 필요한 응답을 선택하세요.